In [27]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#import psycopg2
#from config import db_password
import time
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf

In [28]:
dpo_table = pd.read_csv('dpo_table.csv')
#pkg_dim_table = pd.read_csv('pkg_dims_table.csv')
#region = pd.read_csv('region_table.csv')

In [29]:
for column in dpo_table.columns:
    print(f"Column {column} has {dpo_table[column].isnull().sum()} null values")

Column actdely_dt has 0 null values
Column omstracknum has 0 null values
Column ordnum has 0 null values
Column origin_zip has 0 null values
Column dest_zip has 0 null values
Column shipnode has 0 null values
Column channel has 0 null values
Column carrier_scac has 0 null values
Column reqship_dt has 0 null values
Column sysship_dttm has 0 null values
Column sysship_dt has 0 null values
Column estdely_dt has 0 null values
Column actdely_month has 0 null values
Column estdely_week has 0 null values
Column delivered_late has 0 null values
Column xcpnscan1_desc has 0 null values
Column xcpnscan2_desc has 0 null values
Column xcpnscan3_desc has 0 null values
Column xcpnscan4_desc has 0 null values
Column planned_transit has 0 null values
Column carrier_pub_tnt has 0 null values
Column units has 0 null values
Column transit_override has 0 null values
Column carrier_xcp has 0 null values
Column guest_xcp has 0 null values
Column holiday_xcp has 0 null values
Column weather_xcp has 0 null val

In [30]:
dpo_table.head()

,actdely_dt,omstracknum,ordnum,origin_zip,dest_zip,shipnode,channel,carrier_scac,reqship_dt,sysship_dttm,...,node_on_time_ship,ots_rsd,post_release_inf,node_mnr,carrier_not_meet_tnt,carrier_defect,origin_defect,system_defect,uncontrollable_defect,dpo_delivered_late
0,2019-02-03,1Z2E24F1YW37836926,1022331943153,17408,60126,9475,FC 9475,UPSN,2019-01-30,2019-01-29 20:25,...,NaN,0.0,0.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N
1,2019-02-03,420100169261290982827026490071,9250073345592,7047,10016,STR1865,STR,USPS,2019-02-01,2019-02-01 15:51,...,Node: On Time Ship,0.0,0.0,NaN,NaN,NaN,origin,NaN,NaN,N
2,2019-02-03,420100259261290982827019489617,3550521247344,7047,10025,STR1865,STR,USPS,2019-02-02,2019-02-02 12:08,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,N
3,2019-02-03,420100269261290982827013614626,9050522892943,7047,10026,STR1865,STR,USPS,2019-02-02,2019-02-02 13:21,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,N
4,2019-02-03,420100289261290203606120962697,1050507158738,10451,10028,STR2475,STR,USPS,2019-02-01,2019-01-31 17:22,...,NaN,0.0,0.0,Node: Manifested - not received,NaN,NaN,origin,NaN,NaN,N


In [31]:
dpo_table=dpo_table.fillna(0)

In [32]:
dpo_table.dtypes

actdely_dt                object
omstracknum               object
ordnum                     int64
origin_zip                 int64
dest_zip                   int64
shipnode                  object
channel                   object
carrier_scac              object
reqship_dt                object
sysship_dttm              object
sysship_dt                object
estdely_dt                object
actdely_month             object
estdely_week               int64
delivered_late            object
xcpnscan1_desc            object
xcpnscan2_desc            object
xcpnscan3_desc            object
xcpnscan4_desc            object
planned_transit          float64
carrier_pub_tnt          float64
units                    float64
transit_override          object
carrier_xcp               object
guest_xcp                 object
holiday_xcp               object
weather_xcp               object
uncontrol_xcp             object
order_dttm                object
late_ship_business        object
node_on_ti

In [33]:
dpo_table = dpo_table.drop(['ordnum','reqship_dt','delivered_late','sysship_dttm','sysship_dt','actdely_month','xcpnscan1_desc','xcpnscan4_desc','xcpnscan2_desc','xcpnscan3_desc','carrier_pub_tnt','transit_override','order_dttm','late_ship_business','node_on_time_ship','carrier_defect','origin_defect','system_defect','uncontrollable_defect' ], axis=1)

In [34]:
dpo_table.head()

,actdely_dt,omstracknum,origin_zip,dest_zip,shipnode,channel,carrier_scac,estdely_dt,estdely_week,planned_transit,...,carrier_xcp,guest_xcp,holiday_xcp,weather_xcp,uncontrol_xcp,ots_rsd,post_release_inf,node_mnr,carrier_not_meet_tnt,dpo_delivered_late
0,2019-02-03,1Z2E24F1YW37836926,17408,60126,9475,FC 9475,UPSN,2019-02-06,201901,3.0,...,N,N,N,N,N,0.0,0.0,Node: Manifested - not received,0,N
1,2019-02-03,420100169261290982827026490071,7047,10016,STR1865,STR,USPS,2019-02-04,201901,1.0,...,N,N,N,N,N,0.0,0.0,0,0,N
2,2019-02-03,420100259261290982827019489617,7047,10025,STR1865,STR,USPS,2019-02-05,201901,1.0,...,N,N,N,N,N,1.0,1.0,0,0,N
3,2019-02-03,420100269261290982827013614626,7047,10026,STR1865,STR,USPS,2019-02-05,201901,1.0,...,N,N,N,N,N,0.0,0.0,0,0,N
4,2019-02-03,420100289261290203606120962697,10451,10028,STR2475,STR,USPS,2019-02-04,201901,1.0,...,N,N,N,N,N,0.0,0.0,Node: Manifested - not received,0,N


In [35]:
dpo_table.guest_xcp.value_counts()

N    48
Y     2
Name: guest_xcp, dtype: int64

In [36]:
# do NaN's mean the carrier did meet TNT? 
dpo_table.carrier_not_meet_tnt.value_counts()

0                           43
Carrier Did not meet TNT     7
Name: carrier_not_meet_tnt, dtype: int64

In [37]:
dpo_table.dpo_delivered_late.value_counts()

N    36
Y    14
Name: dpo_delivered_late, dtype: int64

In [38]:

dpo_table['actdely_dt'] = pd.to_datetime(dpo_table['actdely_dt'])

In [39]:
dpo_table['estdely_dt'] = pd.to_datetime(dpo_table['estdely_dt'])

In [40]:
dpo_table['estdely_dt'] =dpo_table['estdely_dt'].dt.day_name()


In [41]:
dpo_table.dtypes

actdely_dt              datetime64[ns]
omstracknum                     object
origin_zip                       int64
dest_zip                         int64
shipnode                        object
channel                         object
carrier_scac                    object
estdely_dt                      object
estdely_week                     int64
planned_transit                float64
units                          float64
carrier_xcp                     object
guest_xcp                       object
holiday_xcp                     object
weather_xcp                     object
uncontrol_xcp                   object
ots_rsd                        float64
post_release_inf               float64
node_mnr                        object
carrier_not_meet_tnt            object
dpo_delivered_late              object
dtype: object

In [42]:
dpo_not_dummies=dpo_table.drop(['channel','carrier_scac','carrier_xcp','guest_xcp','holiday_xcp','weather_xcp','uncontrol_xcp','ots_rsd','post_release_inf','node_mnr','carrier_not_meet_tnt'],axis=1)

In [45]:
dpo_not_dummies.dtypes

actdely_dt            datetime64[ns]
omstracknum                   object
origin_zip                     int64
dest_zip                       int64
shipnode                      object
estdely_dt                    object
estdely_week                   int64
planned_transit              float64
units                        float64
dpo_delivered_late            object
dtype: object

In [50]:
dpo_dummies=dpo_table.drop(['actdely_dt','omstracknum','origin_zip','dest_zip','shipnode','estdely_dt','estdely_week','planned_transit','units','dpo_delivered_late'],axis=1)

In [51]:
dpo_dummies.dtypes

channel                  object
carrier_scac             object
carrier_xcp              object
guest_xcp                object
holiday_xcp              object
weather_xcp              object
uncontrol_xcp            object
ots_rsd                 float64
post_release_inf        float64
node_mnr                 object
carrier_not_meet_tnt     object
dtype: object

In [58]:
dpo_dummies=pd.get_dummies(dpo_dummies)

In [59]:
total_dpo_table=pd.concat([dpo_not_dummies,dpo_dummies],axis=1)

In [60]:
total_dpo_table

,actdely_dt,omstracknum,origin_zip,dest_zip,shipnode,estdely_dt,estdely_week,planned_transit,units,dpo_delivered_late,...,carrier_xcp_N,guest_xcp_N,guest_xcp_Y,holiday_xcp_N,weather_xcp_N,uncontrol_xcp_N,node_mnr_0,node_mnr_Node: Manifested - not received,carrier_not_meet_tnt_0,carrier_not_meet_tnt_Carrier Did not meet TNT
0,2019-02-03,1Z2E24F1YW37836926,17408,60126,9475,Wednesday,201901,3.0,3.0,N,...,1,1,0,1,1,1,0,1,1,0
1,2019-02-03,420100169261290982827026490071,7047,10016,STR1865,Monday,201901,1.0,2.0,N,...,1,1,0,1,1,1,1,0,1,0
2,2019-02-03,420100259261290982827019489617,7047,10025,STR1865,Tuesday,201901,1.0,1.0,N,...,1,1,0,1,1,1,1,0,1,0
3,2019-02-03,420100269261290982827013614626,7047,10026,STR1865,Tuesday,201901,1.0,3.0,N,...,1,1,0,1,1,1,1,0,1,0
4,2019-02-03,420100289261290203606120962697,10451,10028,STR2475,Monday,201901,1.0,1.0,N,...,1,1,0,1,1,1,0,1,1,0
5,2019-02-03,420100319261290203606117430789,10451,10031,STR2475,Monday,201901,1.0,1.0,N,...,1,1,0,1,1,1,0,1,1,0
6,2019-02-03,420100659261290203606112330947,10451,10065,STR2475,Tuesday,201901,1.0,7.0,N,...,1,1,0,1,1,1,0,1,1,0
7,2019-02-03,420105389261290982830516198944,10977,10538,STR1808,Tuesday,201901,1.0,1.0,N,...,1,1,0,1,1,1,1,0,1,0
8,2019-02-03,420191439274890982839321335236,19150,19143,STR2527,Monday,201901,1.0,1.0,N,...,1,1,0,1,1,1,1,0,1,0
9,2019-02-03,420208329261290203606325572608,20878,20832,STR1193,Monday,201901,1.0,1.0,N,...,1,1,0,1,1,1,1,0,1,0


In [61]:
total_dpo_table.dtypes

actdely_dt                                       datetime64[ns]
omstracknum                                              object
origin_zip                                                int64
dest_zip                                                  int64
shipnode                                                 object
estdely_dt                                               object
estdely_week                                              int64
planned_transit                                         float64
units                                                   float64
dpo_delivered_late                                       object
ots_rsd                                                 float64
post_release_inf                                        float64
channel_FC 9434                                           uint8
channel_FC 9475                                           uint8
channel_FC 9479                                           uint8
channel_STR                             

In [78]:
y = total_dpo_table['dpo_delivered_late']
X = total_dpo_table.drop(columns = ['dpo_delivered_late','actdely_dt','omstracknum','estdely_dt','shipnode'])

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [80]:
X_train.shape

(37, 25)

In [115]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=100,
   random_state=1)

In [116]:
X.dtypes

origin_zip                                         int64
dest_zip                                           int64
estdely_week                                       int64
planned_transit                                  float64
units                                            float64
ots_rsd                                          float64
post_release_inf                                 float64
channel_FC 9434                                    uint8
channel_FC 9475                                    uint8
channel_FC 9479                                    uint8
channel_STR                                        uint8
carrier_scac_FDXG                                  uint8
carrier_scac_UPSM                                  uint8
carrier_scac_UPSN                                  uint8
carrier_scac_USPS                                  uint8
carrier_xcp_N                                      uint8
guest_xcp_N                                        uint8
guest_xcp_Y                    

In [117]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [118]:
y_pred = classifier.predict(X_test)

y_pred

array(['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'],
      dtype=object)

In [119]:
y_pred=pd.DataFrame(y_pred)

In [120]:
y_pred.value_counts()

N    13
dtype: int64

In [121]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.6923076923076923
